In [1]:
import pandas as pd
import numpy as np
from sklearn.decomposition.pca import PCA

In [2]:
data = pd.read_csv('iris.csv')
data.head()

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [56]:
mean = np.mean(data)
x = data - mean

# iterate over number of principal components
n_comp = 2
for i in range(n_comp):
    u_list = []
    steps = 1000
    w = np.random.rand(150,1).reshape(-1)
    prev_w = 0
    threshold = 1e-6
    for i in range(steps):
        # set u
        u = np.dot(w.T, x) / np.dot(w.T,w)
        # normalize u
        u = u / np.sqrt(np.dot(u,u.T))
        # get w
        w = np.dot(x, u) / np.sqrt(np.dot(u,u.T))

        u_list.append(u)

        # convergence check
        if i != 0 and i %10 ==0:
            if prev_w - np.linalg.norm(w) > threshold:
                prev_w = np.linalg.norm(w)
            else:
                break

    recon = np.dot(w.reshape(-1,1), u.reshape(1,-1))
    print(u)
    wu = np.dot(w.reshape(-1,1), u.reshape(1,-1))
    x = x - wu

[ 0.36138659 -0.08452251  0.85667061  0.3582892 ]
[-0.65659765 -0.73015232  0.17337383  0.07548934]


In [61]:
def NIPALS(data, n_comp=2):

    # mean center
    mean = np.mean(data)
    x = data - mean
    
    # return values
    u_list = []
    w_list = []
    # iterate over number of principal components
    for i in range(n_comp):
        steps = 1000
        w = np.random.rand(150,1).reshape(-1)
        prev_w = 0
        threshold = 1e-6
        
        # repeat until convergence
        for i in range(steps):
            # set u
            u = np.dot(w.T, x) / np.dot(w.T,w)
            # normalize u
            u = u / np.sqrt(np.dot(u,u.T))
            # get w
            w = np.dot(x, u) / np.sqrt(np.dot(u,u.T))

            # convergence check
            if i != 0 and i %10 ==0:
                if prev_w - np.linalg.norm(w) > threshold:
                    prev_w = np.linalg.norm(w)
                else:
                    break
        
        u_list.append(u)
        w_list.append(w)
        recon = np.dot(w.reshape(-1,1), u.reshape(1,-1))
        wu = np.dot(w.reshape(-1,1), u.reshape(1,-1))
        x = x - wu
        
    return w_list, u_list

In [64]:
w, u = NIPALS(data)
u

[array([ 0.36138659, -0.08452251,  0.85667061,  0.3582892 ]),
 array([ 0.65659012,  0.73016005, -0.17337284, -0.07548229])]

In [63]:
pca = PCA(n_components=2)
components = pca.fit_transform(data)
pca_recon = pca.inverse_transform(components)
pd.DataFrame(pca_recon)
pca.components_

array([[ 0.36138659, -0.08452251,  0.85667061,  0.3582892 ],
       [ 0.65658877,  0.73016143, -0.17337266, -0.07548102]])